# UNSW-NB15: Preparação da Data

## Carregamento das bibliotecas

In [31]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import pickle
import warnings
warnings.filterwarnings("ignore")

In [32]:
# Carregamento dos datasets
train = pd.read_csv('./train_alldata_preparacao.csv')
test = pd.read_csv('./test_alldata_preparacao.csv')

In [33]:
# Retirando a coluna 'attack_cat' que não terá relevancia nesta parte do problema

train = train.drop(columns='attack_cat')
test = test.drop(columns='attack_cat')

print(train.columns)
print()
print(test.columns)
print()

Index(['dur', 'spkts', 'dpkts', 'sbytes', 'dbytes', 'rate', 'sttl', 'dttl',
       'dload', 'dloss',
       ...
       'state_CON', 'state_ECO', 'state_FIN', 'state_INT', 'state_PAR',
       'state_REQ', 'state_RST', 'state_URN', 'state_no', 'label'],
      dtype='object', length=178)

Index(['dur', 'spkts', 'dpkts', 'sbytes', 'dbytes', 'rate', 'sttl', 'dttl',
       'dload', 'dloss',
       ...
       'state_CLO', 'state_CON', 'state_ECO', 'state_FIN', 'state_INT',
       'state_PAR', 'state_REQ', 'state_RST', 'state_URN', 'state_no'],
      dtype='object', length=178)



In [34]:
# Funções uteis
def multi_corr(col1, col2="label", df=train):
    '''
    CORRELAÇAO + LOG CORRELAÇAO
    A função calcula o coeficiente de correlação de Pearson entre col1 e col2 usando o método corr() e guarda o valor numa variável chamada corr.
    A função então aplica a função np.log1p aos valores em col1.
        A função np.log1p() é usada para calcular o logaritmo natural [1 + col1].
        [A transformação de log pode ser útil para reduzir
        o impacto de outliers ou valores extremos nos dados
        e pode tornar a relação entre as duas variáveis mais
        linear ou mais fácil de modelar.] 
    Calcula a correlação entre col1 transformada e col2 e guarda o valor numa variável chamada log_corr.
    Por fim, a função imprime os valores de corr e log_corr.
    '''

    if df[col2].dtype == 'object':
        dummy_df = pd.get_dummies(df[col2])
        corr = df[col1].corr(dummy_df.iloc[:, 0])
        log_corr = df[col1].apply(np.log1p).corr(dummy_df.iloc[:, 0])
    else:
        corr = df[col1].corr(df[col2])
        log_corr = df[col1].apply(np.log1p).corr(df[col2])

    print("Correlation: {}\nlog_Correlation: {}".format(corr, log_corr))

    if log_corr > corr:
        return col1
    else:
        return None

In [35]:
def corr(col1, col2="label", df=train):
    """
    CORRELAÇAO
    A função calcula o coeficiente de correlação de Pearson entre col1 e col2 e guarda o valor numa variável chamada corr.
    Devolve o valor de corr.
    """
    return df[[col1, col2]].corr().iloc[0,1]

## Remoção das colunas com correlações elevadas

In [36]:
# Seleção das colunas com correlações elevadas relativemante a outras
# Referência: https://chrisalbon.com/machine_learning/feature_selection/drop_highly_correlated_features/
corr_matrix = train.corr().abs()

# Selecionamos apenas o triangulo superior da matrix de correlação
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Encontra o index das colunas com uma correlação maior que 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

In [37]:
# Não queremos usar estas colunas nos gráficos porque têm alta corr
# E provavelmente têm o mesmo tipo de comportamento que uma coluna já traçada num gráfico
print(to_drop)

['dbytes', 'dloss', 'synack', 'state_FIN']


In [38]:
# Irá ser mantida a coluna 'state_FIN', pois o seu comportamento está relacionado com as colunas 'state' e será util na construção dos classificadores

to_drop.remove('state_FIN')
print(to_drop)

['dbytes', 'dloss', 'synack']


In [39]:
# Irão ser removidas as colunas do dataset de treino e teste

train.drop(columns=to_drop, inplace=True)
test.drop(columns=to_drop, inplace=True)

In [40]:
train.shape, test.shape

((238682, 175), (82332, 175))

## Aplicar log1p nas Colunas numéricas

        A função np.log1p() é usada para calcular o log [1 + col1].
        [A transformação de log pode ser útil para reduzir
        o impacto de outliers ou valores extremos nos dados
        e pode tornar a relação entre as duas variáveis mais
        linear ou mais fácil de modelar.] 

Durante  Parte 2, descobrimos que poucas colunas numéricas mostram uma melhor visualização para curvas PDF((Probability Density Function)) se aplicarmos log1p às colunas.

Irá ser realizado log1p em todas as colunas e verificar o valor de correlação da coluna original e da coluna log1p com a coluna de alvo ("label")

In [41]:
# Obtendo o número de valores únicos de todas as colunas
# Se os valores únicos forem altos, isso significa que tem um conjunto contínuo de valores (indicador de coluna numérica)
col_unique_values = train.nunique()

In [42]:
# Se os valores unicos forem maiores que um determinado limite, iremos verificar a sua correção
col = col_unique_values[col_unique_values>200].index
print(col)

Index(['dur', 'spkts', 'dpkts', 'sbytes', 'rate', 'dload', 'sinpkt', 'dinpkt',
       'djit', 'tcprtt', 'ackdat', 'smean', 'dmean'],
      dtype='object')


In [43]:
columns_with_better_log_corr = []
for column in col:
    print("{:-^30}".format(column))
    result = multi_corr(column)
    if result is not None:
        columns_with_better_log_corr.append(result)

log1p_col = columns_with_better_log_corr

print("Colunas com melhor correlação log: {}".format(columns_with_better_log_corr))

-------------dur--------------
Correlation: 0.04332431396943613
log_Correlation: 0.02723693722683262
------------spkts-------------
Correlation: -0.06224721238306698
log_Correlation: -0.36190591678361356
------------dpkts-------------
Correlation: -0.12670761233602287
log_Correlation: -0.4882593880897682
------------sbytes------------
Correlation: 0.023286407758299998
log_Correlation: -0.3137630961055886
-------------rate-------------
Correlation: 0.3968635761188683
log_Correlation: 0.3230321454284616
------------dload-------------
Correlation: -0.34525528355118434
log_Correlation: -0.645083946659408
------------sinpkt------------
Correlation: -0.15159363512221158
log_Correlation: -0.12168088912205516
------------dinpkt------------
Correlation: -0.02019321097231751
log_Correlation: -0.026365401266160517
-------------djit-------------
Correlation: -0.05638419009047023
log_Correlation: -0.18584620147932981
------------tcprtt------------
Correlation: 0.08547798233545927
log_Correlation: 0

In [44]:
def log1p_transform(col, df=train):
    '''
    Esta função aplica a função np.log1p() a uma determinada coluna de um dataframe. 
    De seguida, cria uma nova coluna com o nome da coluna original anexado com _log1p e guarda os valores transformados em log nessa nova coluna.
    Por fim, remove a coluna original do DataFrame.
    '''
    new_col = col+'_log1p'
    df[new_col] = df[col].apply(np.log1p)
    df.drop(col, axis=1, inplace=True)

In [45]:
# Transformando as colunas numéricas com log1p nos conjuntos treino e teste
for col in log1p_col:
    log1p_transform(col, df=train)
    log1p_transform(col, df=test)

print(train.shape, test.shape)
print()
print(train.columns)
print()
print(test.columns)

(238682, 175) (82332, 175)

Index(['dur', 'spkts', 'dpkts', 'sbytes', 'rate', 'sttl', 'dttl', 'dload',
       'dinpkt', 'djit',
       ...
       'state_INT', 'state_PAR', 'state_REQ', 'state_RST', 'state_URN',
       'state_no', 'label', 'sinpkt_log1p', 'tcprtt_log1p', 'ackdat_log1p'],
      dtype='object', length=175)

Index(['dur', 'spkts', 'dpkts', 'sbytes', 'rate', 'sttl', 'dttl', 'dload',
       'dinpkt', 'djit',
       ...
       'state_FIN', 'state_INT', 'state_PAR', 'state_REQ', 'state_RST',
       'state_URN', 'state_no', 'sinpkt_log1p', 'tcprtt_log1p',
       'ackdat_log1p'],
      dtype='object', length=175)


In [46]:
# Separação dos conjuntos de dados de treino e teste em variáveis ​​de recursos (x_train, x_test) e de destino (y_train, y_test).
# A função drop é usada para remover a coluna de rótulo/alvo dos conjuntos de dados e armazená-los em y_train e y_test, enquanto as colunas restantes são armazenadas em x_train e x_test.

x_train, y_train = train.drop(columns=['label']), train['label']
x_test, y_test = test.drop(columns=['label']), test['label']

In [47]:
print(x_train.shape, y_train.shape)
print()
print(x_test.shape, y_test.shape)

(238682, 174) (238682,)

(82332, 174) (82332,)


In [48]:
x_train.head()

dur  spkts  dpkts  sbytes           rate  sttl  dttl        dload  \
0  0.399296     10      6     614      37.566118   254   252  4487.898926   
1  0.000010      2      0    1496  100000.002500   254     0     0.000000   
2  1.117953     16     12     896      24.151285   254   252  4479.616211   
3  0.859372     10      8     844      19.781887    62   252  9141.558594   
4  0.000010      2      0     956  100000.002500   254     0     0.000000   

       dinpkt        djit  ...  state_FIN  state_INT  state_PAR  state_REQ  \
0   73.886602  109.398531  ...        1.0        0.0        0.0        0.0   
1    0.000000    0.000000  ...        0.0        1.0        0.0        0.0   
2   98.116180  155.654219  ...        1.0        0.0        0.0        0.0   
3  109.906859  183.879469  ...        1.0        0.0        0.0        0.0   
4    0.000000    0.000000  ...        0.0        1.0        0.0        0.0   

   state_RST  state_URN  state_no  sinpkt_log1p  tcprtt_log1p  ackdat_log1p  
0        0.0        0.0       0.0      3.774382      0.104932      0.077684  
1        0.0        0.0       0.0      0.009950      0.000000      0.000000  
2        0.0        0.0       0.0      4.313228      0.095487      0.059709  
3        0.0        0.0       0.0      4.569396      0.148448      0.068482  
4        0.0        0.0       0.0      0.009950      0.000000      0.000000  

[5 rows x 174 columns]

## Normalização Z-Score / Standardizing
    
Como vimos, o intervalo de valores de algumas colunas neste conjunto de dados é muito grande. <br>
Assim sendo, vamos colocar tudo num determinado intervalo usando standardscaler. Depois deste processo, todos os recursos terão média 0 e um desvio padrão de 1

In [49]:
col_unique_values = x_train.nunique()
num_col = col_unique_values[col_unique_values>200].index
print(num_col)

Index(['dur', 'spkts', 'dpkts', 'sbytes', 'rate', 'dload', 'dinpkt', 'djit',
       'smean', 'dmean', 'sinpkt_log1p', 'tcprtt_log1p', 'ackdat_log1p'],
      dtype='object')


In [50]:
# Verificar as colunas binárias no DataFrame
binary_columns = []
for column in train.columns:
    unique_values = train[column].unique()
    if len(unique_values) == 2 and 0 in unique_values and 1 in unique_values:
        binary_columns.append(column)

print(binary_columns)

['proto_3pc', 'proto_a/n', 'proto_aes-sp3-d', 'proto_any', 'proto_argus', 'proto_aris', 'proto_arp', 'proto_ax.25', 'proto_bbn-rcc', 'proto_bna', 'proto_br-sat-mon', 'proto_cbt', 'proto_cftp', 'proto_chaos', 'proto_compaq-peer', 'proto_cphb', 'proto_cpnx', 'proto_crtp', 'proto_crudp', 'proto_dcn', 'proto_ddp', 'proto_ddx', 'proto_dgp', 'proto_egp', 'proto_eigrp', 'proto_emcon', 'proto_encap', 'proto_etherip', 'proto_fc', 'proto_fire', 'proto_ggp', 'proto_gmtp', 'proto_gre', 'proto_hmp', 'proto_i-nlsp', 'proto_iatp', 'proto_ib', 'proto_icmp', 'proto_idpr', 'proto_idpr-cmtp', 'proto_idrp', 'proto_ifmp', 'proto_igmp', 'proto_igp', 'proto_il', 'proto_ip', 'proto_ipcomp', 'proto_ipcv', 'proto_ipip', 'proto_iplt', 'proto_ipnip', 'proto_ippc', 'proto_ipv6', 'proto_ipv6-frag', 'proto_ipv6-no', 'proto_ipv6-opts', 'proto_ipv6-route', 'proto_ipx-n-ip', 'proto_irtp', 'proto_isis', 'proto_iso-ip', 'proto_iso-tp4', 'proto_kryptolan', 'proto_l2tp', 'proto_larp', 'proto_leaf-1', 'proto_leaf-2', 'proto

In [51]:
num_col = list(set(x_train.columns) - set(binary_columns))
print(num_col)

['dpkts', 'ackdat_log1p', 'rate', 'smean', 'dmean', 'spkts', 'tcprtt_log1p', 'dur', 'ct_dst_src_ltm', 'sbytes', 'djit', 'dttl', 'sttl', 'ct_state_ttl', 'state_ACC', 'sinpkt_log1p', 'dload', 'state_CLO', 'dinpkt']


In [52]:
# Aplicar a normalização
scaler = StandardScaler()
scaler = scaler.fit(x_train[num_col])   # ajustar o scaler às colunas numéricas (col) de x_train

x_train[num_col] = scaler.transform(x_train[num_col])   # transforma as colunas numéricas de x_train usando o scaler
x_test[num_col] = scaler.transform(x_test[num_col])   # transforma as colunas numéricas de x_train usando o scaler

x_train.head()

dur     spkts     dpkts    sbytes      rate      sttl      dttl  \
0 -0.141207 -0.106949 -0.163653 -0.046447 -0.490018  0.931403  1.685166   
1 -0.206954 -0.173101 -0.218897 -0.040564  0.173258  0.931403 -0.719469   
2 -0.022872 -0.057335 -0.108409 -0.044566 -0.490107  0.931403  1.685166   
3 -0.065450 -0.106949 -0.145238 -0.044913 -0.490136 -0.818199  1.685166   
4 -0.206954 -0.173101 -0.218897 -0.044166  0.173258  0.931403 -0.719469   

      dload    dinpkt      djit  ...  state_FIN  state_INT  state_PAR  \
0 -0.349843 -0.019186 -0.124910  ...        1.0        0.0        0.0   
1 -0.351351 -0.084103 -0.147933  ...        0.0        1.0        0.0   
2 -0.349846  0.002102 -0.115176  ...        1.0        0.0        0.0   
3 -0.348279  0.012461 -0.109236  ...        1.0        0.0        0.0   
4 -0.351351 -0.084103 -0.147933  ...        0.0        1.0        0.0   

   state_REQ  state_RST  state_URN  state_no  sinpkt_log1p  tcprtt_log1p  \
0        0.0        0.0        0.0       0.0      0.713553      0.994994   
1        0.0        0.0        0.0       0.0     -0.785472     -0.508218   
2        0.0        0.0        0.0       0.0      0.928126      0.859683   
3        0.0        0.0        0.0       0.0      1.030134      1.618382   
4        0.0        0.0        0.0       0.0     -0.785472     -0.508218   

   ackdat_log1p  
0      1.604553  
1     -0.481801  
2      1.121812  
3      1.357409  
4     -0.481801  

[5 rows x 174 columns]

In [53]:
file_path = 'dataPronta/'

x_train.shape, x_test.shape

((238682, 174), (82332, 174))

In [54]:
# Verificar se as colunas de treino e teste são as mesmas
all(x_train.columns == x_test.columns)

True

In [55]:
# Guardar todos os dados para usar no treino e teste das redes neuronais
pickle.dump((x_train, y_train), open('./dataPronta/final_train.pkl', 'wb'))
pickle.dump((x_test, y_test), open('./dataPronta/final_test.pkl', 'wb'))

In [56]:
x_test.shape, y_test.shape, x_train.shape, y_train.shape

((82332, 174), (82332,), (238682, 174), (238682,))